In [ ]:
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1617950361449_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
import boto3
dynamodb=boto3.client('dynamodb',region_name='ap-southeast-1')
s3=boto3.client('s3',region_name='ap-southeast-1')
athena=boto3.client('athena',region_name='ap-southeast-1')
sqs=boto3.client('sqs',region_name='ap-southeast-1')
queueUrl='https://sqs.ap-southeast-1.amazonaws.com/855696220043/floatingShelf_exec_sql'
tableNm='AnalysisReportSql-v6q6okep3bebnkt4bszyldyyie-hanpeng'
sqlList=[]
execList=[]
def getsqlDetail(bucket,path):
    response = s3.get_object(
        Bucket=bucket,
        Key=path,
    )
    return response["Body"].read().decode('utf-8') if response["Body"] is not None else ''
def execAthenaSql(sql,tableNm):
    s3_output="s3://rupiahplus-data-warehouse/etl/floatingshelf/"+tableNm
    response = athena.start_query_execution(
        QueryString=sql,
        QueryExecutionContext={
            'Database': "han_test"
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    return {"tableNm":tableNm,"execId":response["QueryExecutionId"]}
def getRecord(record):
    sqlbucket=record["sqlbucket"]["S"]
    sqlpath=record["sqlpath"]["S"]
#     获取sql内容
    sqlDetail=getsqlDetail(sqlbucket,sqlpath)
    mobj={}
    mobj['sqlbucket']=sqlbucket
    mobj['sqlpath']=sqlpath
    mobj['tableNm']=record["tableNm"]["S"]
    mobj['executecycle']=record["executecycle"]["S"]
    mobj['sqlDetail']=sqlDetail
    return mobj
def getsqlList():
    response = dynamodb.scan(
        TableName=tableNm,
        Limit=1000,
        Select='ALL_ATTRIBUTES',
#         FilterExpression=" executecycle =:executecycle",
        #             ':executecycle': {
#                 'S': 'DAILY'
#             }
        FilterExpression="tbtype =:tbtype",
        ExpressionAttributeValues={
             ':tbtype': {
                'S': 'ID_CREDINEX'
            }
        }
    )
    sqlList=list(map(getRecord,response["Items"]))
    return sqlList
# 发送集合到sqs
def sendsqlDataToSqs():
    # 获取sql目录集合
    sqlList=getsqlList()
    if len(sqlList)>0:
        for i in sqlList:
            print(i)
            response = sqs.send_message(
                QueueUrl=queueUrl,
                MessageBody=str(i)
            )
            print(response)
sendsqlDataToSqs()            
# 执行sql
# def refreshQs():
#     # 获取sql目录集合
#     sqlList=getsqlList()
#     print(sqlList)
#     sqllength=len(sqlList)
#     i = 0
#     countNum = 0
#     while True:
#         print(sqlList[i * 5:(i + 1) * 5 if (i + 1) * 5 < sqllength else sqllength])
# #         countNum = execute(
# #             sqlList[i * 5:(i + 1) * 5 if (i + 1) * 5 < len(sqllength) else len(sqllength)], countNum)
#         i = i + 1
#         if countNum >= sqllength:
#             break
        # 睡5分钟
#         time.sleep(100)
# refreshQs()
# for i in sqlList:
#     print(i)
#     sql=getsqlDetail(i["sqlbucket"],i["sqlpath"])
#     execRestult=execAthenaSql(sql,i["tableNm"])
#     execList.append(execRestult)
# print(execList)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'sqlbucket': 'floatingshelf114234-hanpeng', 'sqlpath': 'private/athenaSQl/ID_credinex_loan_review_loan_review_manual_overdue.txt', 'tableNm': 'ID_credinex_loan_review_loan_review_manual_overdue', 'executecycle': 'DAILY', 'sqlDetail': 'with first_bill as\n(\n  select cast(de.customer_id as bigint) as user_id,case when de.repayment_date <= date(now()) then 1 else 0 end as is_expiry,\n  case when de.overdue_amount > 0 then 1 else 0 end as is_overdue\n  from\n  (\n    SELECT customer_id,min(id) as fist_bill_id\n    FROM "credinex_account_etl_s3"."t_credit_bill"\n    where capital_amount > 0  \n    group by customer_id\n  )fb\n  left join "credinex_account_etl_s3"."t_credit_bill" de\n  on fb.fist_bill_id = de.id\n),\npay as \n(\n  SELECT cast(customer_id as bigint) as user_id,count(id) as comsume_count\n  FROM "credinex_account_etl_s3"."t_payment_order"\n  where business_type=\'CONSUMPTION\'\n  group by customer_id\n),\ncase_detail as\n(\n  select flow.id,flow.next_status,info.user_id,\n  

In [ ]:
import boto3
import ast
import time
from datetime import datetime
sqs=boto3.client('sqs',region_name='ap-southeast-1')
s3=boto3.client('s3',region_name='ap-southeast-1')
athena=boto3.client('athena',region_name='ap-southeast-1')
dynamodb=boto3.client('dynamodb',region_name='ap-southeast-1')
QueueUrl='https://sqs.ap-southeast-1.amazonaws.com/855696220043/floatingShelf_exec_sql'
resultBucket="rupiahplus-data-warehouse"
resultpath="/etl/floatingshelf/"

# 更新dynamodb的状态
def updateSqlState(execResult):
    tableId = execResult['tableId']
    sqlexecid=execResult['sqlexecid']
    sqlexecdate = execResult['sqlexecdate']
    resultbucket = execResult['resultbucket']
    resultpath = execResult['resultpath']
    response = dynamodb.update_item(
        TableName='AnalysisReportSql-v6q6okep3bebnkt4bszyldyyie-hanpeng',
        Key={"id":{"S":tableId}},
        UpdateExpression="SET  sqlexecid=:sqlexecid,sqlexecdate =:sqlexecdate ,resultbucket=:resultbucket,resultpath=:resultpath",
        ExpressionAttributeValues={
            ':sqlexecid': {'S': sqlexecid},
            ':sqlexecdate': {'S': sqlexecdate},
            ':resultbucket': {'S': resultbucket},
            ':resultpath': {'S': resultpath},
        },
    )
    print(response)
def getsqlDetail(bucket,path):
    response = s3.get_object(
        Bucket=bucket,
        Key=path,
    )
    return response["Body"].read().decode('utf-8') if response["Body"] is not None else ''
def execAthenaSql(sql,tableNm):
    s3_output="s3://rupiahplus-data-warehouse/etl/floatingshelf/"+tableNm
    response = athena.start_query_execution(
        QueryString=sql,
        QueryExecutionContext={
            'Database': "han_test"
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    # 执行完成之后，更新记录
    sqlstate={}
    sqlstate['tableId']=tableNm
    sqlstate['sqlexecid'] = response["QueryExecutionId"]
    sqlstate['sqlexecdate'] =datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    sqlstate['resultbucket'] = resultBucket
    sqlstate['resultpath'] = resultpath+tableNm+"/"+response["QueryExecutionId"]+".csv"
    updateSqlState(sqlstate)
    return {"tableNm":tableNm,"execId":response["QueryExecutionId"]}
def execSql(messagedetail):
    print(messagedetail)
    delFlag=1
    if messagedetail.get("Body") is None:
        return
    body = ast.literal_eval(messagedetail["Body"])
    if body["sqlbucket"] is None \
            or body["sqlpath"] is None \
            or body["tableNm"] is None:
        deleteMessage(messagedetail['ReceiptHandle'])
        return
    # sqlstr=getsqlDetail(body["sqlbucket"],body["sqlpath"])
    if body.get("sqlDetail")is None or body.get("sqlDetail") is None or body.get("sqlDetail") =='':
        deleteMessage(messagedetail['ReceiptHandle'])
        return
    sqlstr = body["sqlDetail"]
    try:
         execAthenaSql(sqlstr,body["tableNm"])
         # 执行完成之后把结果返回DynamoDb
    except Exception as e:
        import traceback
        print("出错了------------------",e)
        print(traceback.format_exc())
        delFlag=0
    # 拿到消息之后删掉消息
    if delFlag==1:
        deleteMessage(messagedetail['ReceiptHandle'])
def deleteMessage(receiptHandle):
    print("执行删除------")
    response=sqs.delete_message(
        QueueUrl=QueueUrl,
        ReceiptHandle=receiptHandle
    )
    print("删除结果------",response)
noneNum=0
while True:
    time.sleep(60)
    response = sqs.receive_message(
        QueueUrl=QueueUrl,
        AttributeNames=[
            'All'
        ],
        MaxNumberOfMessages=10,
        WaitTimeSeconds=20,
    )
    print("数据是---------",response)
    if response.get("Messages") is not None :
        noneNum=0
        print(list(map(execSql,response["Messages"])))
    else:
        noneNum=noneNum+1
    print(noneNum)
    if noneNum>5:
        break

VBox()